<a href="https://colab.research.google.com/github/siddtheshah/vc_modeling/blob/master/category_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install cityhash
import cityhash
import pandas

from copy import deepcopy

     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for cityhash: filename=cityhash-0.2.3.post9-cp36-cp36m-linux_x86_64.whl size=82872 sha256=0e0a9dce2b8557f7e31b05c0893dbff7b3bf0f76aa79cd63a045a7a34f22fa50
  Stored in directory: /root/.cache/pip/wheels/a4/4d/ae/cf0f1f8058ca2738fe972c69291083a0a06765271fb1047edd
Successfully built cityhash


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
org_info = pd.read_csv("/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/organizations.csv")
print(np.shape(org_info))
# Filter down the data purely to investees
org_info = org_info[org_info['primary_role'] == 'company']
print(np.shape(org_info))

(1089733, 41)
(1025256, 41)


In [11]:
import collections

# Select organization data
# Clean the df

comma_sep_list_vars = ['category_list', 'category_groups_list']
other_vars = ['uuid']
org_info_selected = org_info[other_vars + comma_sep_list_vars]
print("Rows to build vocabulary before NA drop: ", np.shape(org_info_selected)[0])
org_info_selected = org_info_selected.dropna()
org_info_selected = org_info_selected.reset_index(drop=True)

vocabulary = collections.Counter()

iter = 0
test_stop = -1
total_rows = np.shape(org_info_selected)[0]
print("Processing ", total_rows, " rows.")
for (i, row) in org_info_selected.iterrows():
  if iter == test_stop:
    break
  iter += 1
  if iter % 100000 == 0:
    print(iter, " of ", total_rows)

  for col, _ in row.iteritems():
    if col in comma_sep_list_vars:
      # Break categories into unigrams
      token_lists = [x.split(' ') for x in row[col].strip().split(',')]
      tokens = []
      for token_list in token_lists:
        for token in token_list:
          tokens.append(token)

      for token in tokens:
        vocabulary[token] += 1

# Show the unfiltered vocabulary counts
print(vocabulary.most_common())

# Filter low count words
low_count_threshold = 15
keys_to_remove = []
for word, count in vocabulary.items():
  if count < low_count_threshold:
    keys_to_remove.append(word)

for key in keys_to_remove:
  del vocabulary[key]

# Now rank and filter by document frequency. The top half frequent words will be dropped
key_list = list(vocabulary.keys())
fraction_to_drop = .25
chop = sorted(key_list, key=lambda x: vocabulary[x])[-int(fraction_to_drop*len(key_list)):]
for key in chop:
  del vocabulary[key]

# Create index to word mapping
word_to_index = {}
word_index = 1 # Leave space for adding the hash key.
for word, count in vocabulary.items():
  word_to_index[word] = word_index
  word_index += 1

print(word_to_index)
vocabulary_size = word_index

Filtered rows to build vocabulary:  963967
Processing  963967  rows.
100000  of  963967
200000  of  963967
300000  of  963967
400000  of  963967
500000  of  963967
600000  of  963967
700000  of  963967
800000  of  963967
900000  of  963967
[('and', 783041), ('Software', 421264), ('Services', 376331), ('Information', 328481), ('Technology', 312525), ('Internet', 239214), ('Marketing', 190913), ('Health', 173889), ('Media', 169091), ('Care', 167984), ('Manufacturing', 167663), ('Entertainment', 134448), ('Advertising', 133623), ('Sales', 124540), ('Financial', 124035), ('Design', 122398), ('Hardware', 121259), ('Mobile', 119113), ('Shopping', 111194), ('Commerce', 104907), ('Real', 104381), ('Estate', 102213), ('Consumer', 98114), ('Apps', 98016), ('Engineering', 88802), ('Education', 86703), ('Science', 77933), ('Electronics', 77801), ('E-Commerce', 75006), ('Data', 73557), ('Analytics', 71562), ('Food', 69375), ('Consulting', 63881), ('Transportation', 61691), ('Energy', 59976), ('Beve

In [12]:
# Show words from our filtered vocabulary.
print(vocabulary.most_common())

[('Location', 4184), ('Beauty', 4155), ('Project', 4081), ('Film', 4044), ('Aerospace', 4017), ('Wholesale', 3853), ('Diagnostics', 3828), ('Sharing', 3770), ('Outsourcing', 3768), ('Hospital', 3760), ('Planning', 3744), ('Life', 3715), ('Email', 3659), ('Mechanical', 3633), ('Rental', 3602), ('Crowdfunding', 3541), ('Cryptocurrency', 3510), ('Local', 3493), ('Therapeutics', 3475), ('Water', 3240), ('Art', 3218), ('Architecture', 3172), ('Wine', 3057), ('And', 3057), ('Spirits', 3057), ('Production', 2905), ('Distribution', 2798), ('Streaming', 2742), ('Advice', 2739), ('Collaboration', 2696), ('Investment', 2624), ('Employment', 2608), ('Packaging', 2587), ('Furniture', 2586), ('Semiconductor', 2568), ('Interior', 2544), ('Electrical', 2532), ('Smart', 2491), ('Database', 2418), ('Document', 2382), ('Risk', 2377), ('Precious', 2363), ('Metals', 2363), ('Augmented', 2363), ('Networking', 2358), ('Language', 2344), ('Support', 2338), ('Drones', 2264), ('Storage', 2229), ('Decor', 2227),

In [23]:
# Verify that org uuid is clean
org_info_selected['uuid'].iloc[4]

'60485007-8856-bbac-aa1b-c535c41f5f47'

In [24]:
from scipy.sparse import coo_matrix
import collections

# Load our filtered vocabulary if we've saved it on disk.

row_indices = []
col_indices = []
data_values = []

used_hashes = set()
lookup_index = []
lookup_value = []

iter = 0
test_stop = -1
total_rows = np.shape(org_info_selected)[0]

print("Processing ", total_rows, " rows.")
for (i, row) in org_info_selected.iterrows():
  if iter == test_stop:
    break
  iter += 1
  if iter % 100000 == 0:
    print(iter, " of ", total_rows)

  uuid = row['uuid'].strip()
  hash = cityhash.CityHash64(uuid)

  if hash in used_hashes:  # Hash collision!
    print(hash)
    continue          # which shouldn't really happen
  used_hashes.add(hash)

  lookup_value.append(uuid)
  lookup_index.append(hash)

  row_indices.append(i)
  col_indices.append(0)
  data_values.append(hash)
    
  for col, _ in row.iteritems():
    if col in comma_sep_list_vars:
      # Break categories into unigrams
      unigram_counts = collections.Counter()
      token_lists = [x.split(' ') for x in row[col].strip().split(',')]
      for token_list in token_lists:
        for token in token_list:
          unigram_counts[token] += 1
      
      for unigram, count in unigram_counts.items():
        if unigram in vocabulary:
          row_indices.append(i)
          col_indices.append(word_to_index[unigram])
          data_values.append(count)

unclustered = coo_matrix((data_values, (row_indices, col_indices)))
print(unclustered.toarray().shape)

Processing  963967  rows.
100000  of  963967
200000  of  963967
300000  of  963967
400000  of  963967
500000  of  963967
600000  of  963967
700000  of  963967
800000  of  963967
900000  of  963967
(963967, 564)


In [26]:
import scipy.sparse
print(unclustered)
## Generate company lookup
lookup = pd.DataFrame.from_dict({'uuid':lookup_index, 'hash':lookup_value})

lookup.to_csv('/lookup.csv')

# category_features = pd.DataFrame.sparse.from_spmatrix(joined)
scipy.sparse.save_npz('/category_features.npz', unclustered)
# category_features.to_pickle('/category_features.npz')



  (0, 0)	1.3685534557686295e+19
  (1, 0)	7.640156219293676e+17
  (1, 1)	1.0
  (1, 2)	1.0
  (2, 0)	1.0846552445983457e+19
  (3, 0)	5.087506707876194e+18
  (3, 3)	1.0
  (4, 0)	9.094535307341386e+18
  (4, 4)	1.0
  (4, 5)	1.0
  (5, 0)	1.7418110891307706e+19
  (5, 1)	1.0
  (6, 0)	7.773244737896805e+18
  (7, 0)	1.5263381640365195e+19
  (8, 0)	2.7762472306795356e+18
  (8, 6)	1.0
  (8, 7)	1.0
  (9, 0)	1.7179139454557108e+19
  (9, 8)	1.0
  (10, 0)	3.143841549007344e+18
  (11, 0)	1.0303043534068977e+18
  (11, 9)	1.0
  (11, 4)	1.0
  (11, 5)	1.0
  (11, 10)	1.0
  :	:
  (963953, 208)	1.0
  (963953, 21)	1.0
  (963953, 33)	1.0
  (963954, 0)	7.398411164059276e+18
  (963954, 112)	1.0
  (963955, 0)	1.2078929441060368e+19
  (963956, 0)	4.0054335165626184e+18
  (963956, 306)	1.0
  (963956, 3)	1.0
  (963957, 0)	1.689286265119951e+19
  (963957, 353)	1.0
  (963958, 0)	7.229716827056183e+18
  (963958, 532)	1.0
  (963959, 0)	4.62656412319939e+18
  (963960, 0)	2.978566027619601e+18
  (963961, 0)	9.85213510384623

In [ ]:
print(joined)

  (0, 0)	1.3685534557686295e+19
  (1, 0)	7.640156219293676e+17
  (2, 0)	1.0846552445983457e+19
  (3, 0)	5.087506707876194e+18
  (4, 0)	9.094535307341386e+18
  (5, 0)	1.7418110891307706e+19
  (6, 0)	7.773244737896805e+18
  (7, 0)	1.5263381640365195e+19
  (8, 0)	2.7762472306795356e+18
  (9, 0)	1.7179139454557108e+19
  (10, 0)	3.143841549007344e+18
  (11, 0)	1.0303043534068977e+18
  (12, 0)	2.4974301446054984e+18
  (13, 0)	5.507971386325512e+18
  (14, 0)	2.8624645959363738e+17
  (15, 0)	1.22265356693808e+19
  (16, 0)	1.1228059662374996e+19
  (17, 0)	1.1671402361252221e+18
  (18, 0)	5.369011697229158e+18
  (19, 0)	1.4750419745579184e+19
  (20, 0)	6.888766334922708e+18
  (21, 0)	1.106971057344432e+19
  (22, 0)	1.615128531752331e+19
  (23, 0)	8.997274164534324e+17
  (24, 0)	1.569457398809743e+19
  :	:
  (808919, 271)	1.0
  (808919, 203)	1.0
  (808919, 381)	1.0
  (808920, 109)	1.0
  (808920, 114)	1.0
  (808922, 39)	1.0
  (808922, 233)	1.0
  (808922, 351)	1.0
  (808923, 348)	1.0
  (808924, 443